In [1]:
from http import client
from urllib import response
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from img2vec_pytorch import Img2Vec
from sentence_transformers import SentenceTransformer
import plotly.express as px
import plotly.graph_objects as go
import urllib.request
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import json
import numpy as np

/home/dustin/anaconda3/envs/web3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Import Data
print('Load Description Embeddings')
desc_emb = np.load('../data/desc_emb.npz')
traindata_desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(traindata_desc_embeddings.shape)

print('Load Description Labels, Address Labels and Name Labels')
NFT_attributes = np.load('../data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']
print(description_list.shape)

train_desc_address = list(zip(contract_addresses, traindata_desc_embeddings, description_list))


Load Description Embeddings
(11083, 768)
Load Description Labels, Address Labels and Name Labels
(11083,)


In [3]:
def find_basketball(key_words, dictionary):
    if type(dictionary) is dict:
        ret_value = 0
        for key in dictionary.keys():
            ret_value += find_basketball(key_words, dictionary[key])
        if ret_value > 0:
            return 1
        return 0
    elif type(dictionary) is str:
        for key in key_words:
            if key in dictionary.lower():
                return 1
    return 0

In [4]:
key_words = ['basketball', 'nba', 'hoops']
counter = 0

In [ ]:
key_words = ['basketball', 'nba', ]
counter = 0
# Only Description
# for description in description_list:
#     for key_word in key_words:
#         if key_word in description.lower():
#             counter += 1
#             break

NFTjson_list = []
for i, address in enumerate(contract_addresses):
    #print(counter)
    # evtl muessen wir checken ob alles erc721 sind und tokenID 1 immer existiert
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={address}&tokenId=1&tokenType=erc721"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
        NFTjson_list.append(NFTjson)
    except:
        continue
    if i % 100 == 0:
        print(f'{i}/{len(contract_addresses)}')



In [11]:
#np.savez('/home/dustin/Documents/Ducia/NFT_Embedding/data/NFTjson_list.npz', NFTjson_list=NFTjson_list)
NFTjson_load = np.load('/home/dustin/Documents/Ducia/NFT_Embedding/data/NFTjson_list.npz',  allow_pickle=True)

In [18]:
d = dict(zip(("NFTjson_list"), (NFTjson_load[k] for k in NFTjson_load)))
print(d.keys())
NFTjson_list = d['N']
print(type(NFTjson_list))

dict_keys(['N'])
<class 'numpy.ndarray'>


In [23]:
print(NFTjson_list[0]['contract']['address'])

0x14232931f4151a7a962c1929f670378159d87fb7


In [27]:
print('Computing Count')
address_list = []
counter = 0
for NFTjson in NFTjson_list:
    bit = find_basketball(key_words, NFTjson)
    counter += bit
    if bit > 0:
        address_list.append(NFTjson['contract']['address'])

print(counter, len(address_list))

'''
keywords: basketball; only description --> Counter = 19 
keywords: basketball,nba; only description --> Counter = 39
keywords: basketball; all attributes --> Counter = 
keywords: basketball,nba; all attributes --> Counter = 233
'''

Computing Count
234 234


'\nkeywords: basketball; only description --> Counter = 19 \nkeywords: basketball,nba; only description --> Counter = 39\nkeywords: basketball; all attributes --> Counter = \nkeywords: basketball,nba; all attributes --> Counter = 233\n'

In [5]:
dictionary = {'key1': {
    'key11': 'basketbal',
    'key12': 'Penis'
},
'key2': 'nba'
}

print(find_basketball(key_words, dictionary))
print(key_words)

{'key1': {'key11': 'basketbal', 'key12': 'Penis'}, 'key2': 'nba'}
{'key11': 'basketbal', 'key12': 'Penis'}
basketbal
Penis
nba
1
['basketball', 'nba']
